# Function Testing Notebook - Joseph

This notebook is meant to test external-facing functions to ensure they are working as expected.

A dedicated test_all_functions.py with unit testing might be better, but this is a good benchmark

In [9]:
import logging
import os
import sys
from pathlib import Path
import json

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from okabeito import black, blue, green, lightblue, orange, purple, red, yellow
from seaborn import axes_style

from pythoneeg import constants, core, visualization


In [10]:
tempdir = Path('../../tmp').resolve()
core.set_temp_directory(tempdir)
print(tempdir)

2025-08-04 14:34:08,573 - INFO - Temporary directory set to /mnt/isilon/marsh_single_unit/PythonEEG/tmp
/mnt/isilon/marsh_single_unit/PythonEEG/tmp


In [11]:
# Set up logger with detailed formatting
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO, stream=sys.stdout, force=True)
logger = logging.getLogger()


In [12]:
logger.setLevel(logging.INFO)
wars = []

animal_ids = [p for p in Path("./test-wars-sox5-7").glob("*") if p.is_dir()]
# bad_animal_ids = [
#     "013122_cohort4_group7_2mice both_FHET FHET(2)",
#     "012322_cohort4_group6_3mice_FMUT___MMUT_MWT MHET",
#     "012322_cohort4_group6_3mice_FMUT___MMUT_MWT MMUT",
#     "011622_cohort4_group4_3mice_MMutOLD_FMUT_FMUT_FWT OLDMMT",
#     "011322_cohort4_group3_4mice_AllM_MT_WT_HET_WT M3",
#     "012322_cohort4_group6_3mice_FMUT___MMUT_MWT FHET"
#     # "060921_Cohort 3_EM1_AM2_GF4 AM2" # this seems fine just very high delta for whatever reason
# ]

# Create empty lists to store data
df = []

for i, animal_id in enumerate(animal_ids):
    json_files = list(Path(animal_id).glob('*.json'))
    if len(json_files) > 1:
        print(f"Multiple json files found for {animal_id}")
        break
    json_file = json_files[0]

    # Load the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)['bad_channels_dict']
        for day in data.keys():
            df.append({"animal" : animal_id.name, 'animalday' : day})

# Create dataframe
df = pd.DataFrame(df)

display(df)


,animal,animalday
0,071321_Cohort 3_AM4_CF1_DF3_FF6 DF3,DF3 FWT Jul-15-2021
1,071321_Cohort 3_AM4_CF1_DF3_FF6 DF3,DF3 FWT Jul-13-2021
2,071321_Cohort 3_AM4_CF1_DF3_FF6 DF3,DF3 FWT Jul-14-2021
3,071321_Cohort 3_AM4_CF1_DF3_FF6 DF3,DF3 FWT Jul-16-2021
4,031722_cohort_6_group1_3mice_MMUT_MMUT_MWT M7,M7 MWT Mar-20-2022
...,...,...
234,031722_cohort_6_group1_3mice_MMUT_MMUT_MWT MMU...,MMUT(1) MMut Mar-17-2022
235,031921_cohort 2 group 5 and group 6 mouse M3 c...,mouse M3 cage3A MHet Mar-20-2021
236,031921_cohort 2 group 5 and group 6 mouse M3 c...,mouse M3 cage3A MHet Mar-19-2021
237,031921_cohort 2 group 5 and group 6 mouse M3 c...,mouse M3 cage3A MHet Mar-22-2021


In [13]:
len(df['animal'].unique())

70

In [14]:
save_folder = '/home/dongjp/Downloads/8-4 fine filtering'
save_files = True
save_folder = Path(save_folder)
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
display(save_folder)

PosixPath('/home/dongjp/Downloads/8-4 fine filtering')

In [15]:
df.to_excel(save_folder / f'{save_folder.name}.xlsx', index=False)